In [11]:
class Variable:
    
    def __init__(self, data):
        self.data = data # 통상값 data
        self.grad = None # grad(미분값) 기울기
        #인스턴스 변수 data,grad 넘파이의 다차원 배열(ndarray)
        
        self.creator = None #인스턴스 변수 creator
    
    #변수 생성 함수
    def set_creator(self, func):
        self.creator = func

In [7]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) #forward 메세드 호출시 건네받은 Variable 인스턴스 유지
        output = Variable(y) # 순전파를 계산하면 output이라는 Variable 인스턴스가 생성됨
        output.set_creator(self) # 출력 변수에 창조자 설정
        
        self.input = input # 입력 변수를 보관함.
        self.output = output #
        
        return output
    
    def forward(self,x):
        raise NotImplementedError()
    
    #미분을 계산하는 역전파
    def backward(self,gy):
        raise NotImplementedError()
    
    

In [12]:
#function을 상속한 구체적 함수에서 역전파 구현
# Square 클래스

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    #미분값 계산
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
        

In [13]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        
        return gx    

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라감.
# assert는 조건문과 비슷함
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [16]:
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져옴
b = C.input # 2. 함수의 입력을 가져옴
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드 호출


B = b.creator # 1. 함수를 가져옴
a = B.input # 2. 함수의 입력을 가져옴
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드 호출

A = a.creator
x = A.input
x.grad = A.backward(a.grad)
#마지막으로 변수 a에서 x로의 역전파


print(x.grad)

3.297442541400256


In [17]:
class Variable:
    
    def __init__(self, data):
        self.data = data # 통상값 data
        self.grad = None # grad(미분값) 기울기
        #인스턴스 변수 data,grad 넘파이의 다차원 배열(ndarray)
        
        self.creator = None #인스턴스 변수 creator
    
    #변수 생성 함수
    def set_creator(self, func):
        self.creator = func
        
    # 반복작업 자동화를 위한 메서드
    def backward(self):
        f = self.creator # 1. 함수를 가져옴.
        if f is not None:
            x = f.input # 2. 함수의 입력을 가져옴
            x.grad = f.backward(self.grad) # 3. 함수의 backward 메서드 호출
            x.backward() # 하나 앞 변수의 backward 메서드 호출함. (재귀)
            
        # 만약 Variable 인스턴스의 creator가 None이면 역전파가 중단됨.
        # 창조주가 없기 때문에 해당 Variable 인스턴스는 함수 바깥에서 생성됨 -> 즉 사용자가 만든 것.
        

In [18]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파
y.grad = np.array(1.0)
y.backward()
#이처럼 변수 y의 backward 메서드 호출시 역전파 자동 진행

print(x.grad)



3.297442541400256


In [20]:
class Variable:
    
    def __init__(self, data):
        self.data = data # 통상값 data
        self.grad = None # grad(미분값) 기울기
        #인스턴스 변수 data,grad 넘파이의 다차원 배열(ndarray)
        
        self.creator = None #인스턴스 변수 creator
    
    #변수 생성 함수
    def set_creator(self, func):
        self.creator = func
        
    # 반복작업 자동화를 위한 메서드
    def backward(self):
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() #함수를 가져옴 pop()메서드 사용
            x,y = f.input, f.output # 함수의 입력과 출력을 가져옴
            x.grad = f.backward(y.grad) # backward 메서드 호출함.
            
            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가함.
            
        
        #f = self.creator # 1. 함수를 가져옴.
        #if f is not None:
            #x = f.input # 2. 함수의 입력을 가져옴
            #x.grad = f.backward(self.grad) # 3. 함수의 backward 메서드 호출
            #x.backward() # 하나 앞 변수의 backward 메서드 호출함. (재귀)
            
            
            
        # 만약 Variable 인스턴스의 creator가 None이면 역전파가 중단됨.
        # 창조주가 없기 때문에 해당 Variable 인스턴스는 함수 바깥에서 생성됨 -> 즉 사용자가 만든 것.

In [21]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파
y.grad = np.array(1.0)
y.backward()
#이처럼 변수 y의 backward 메서드 호출시 역전파 자동 진행

print(x.grad)

#속도 개선됨.


3.297442541400256


In [ ]:
#

x = Variable(np.array(0.5))
f = Square() #
y = f(x)
